In [2]:
import numpy as np
import mdtraj as md
import itertools
import time
import matplotlib.pyplot as plt
#from numba import jit
#from numba import njit

In [3]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [4]:
np.__version__

'1.20.2'

In [5]:
md.version.full_version

'1.9.6'

In [6]:
## 

In [7]:
def trapezoid(y, dx=1):
    
    tmp = 0
    
    for i in range(0,len(y)-1):
        
        tmp += (y[i+1]+y[i])/2
        
    return tmp*dx

In [20]:
def appendSpherical_np(xyz):
    
    # converting xyz coordinates to the spherical coordinates 
    
    r = np.sqrt(xyz[:,0]**2 + xyz[:,1]**2 + xyz[:,2]**2) # magnitude
    
    r_tmp = np.sqrt(xyz[:,0]**2 + xyz[:,1]**2)
    
    theta = np.arctan2(r_tmp, xyz[:,2])
    phi = np.arctan2(xyz[:,1], xyz[:,0]) # arctan2 considers the sign of x,y while computing the angle
    
    tmp = np.zeros((len(phi),3))
    tmp[:,0] = r
    tmp[:,1] = theta
    tmp[:,2] = phi
    
    return tmp

In [8]:
#@njit
def f1re(rtp):
    
    return np.sin(rtp[:,1]) * np.cos(rtp[:,1]) * np.cos(rtp[:,2]) / (rtp[:,0]**3)

In [9]:
#@njit
def f1im(rtp):
    
    return -np.sin(rtp[:,1]) * np.cos(rtp[:,1]) * np.sin(rtp[:,2]) / (rtp[:,0]**3)

In [10]:
#@njit
def f2re(rtp):
    
    return (np.sin(rtp[:,1])**2) * (np.cos(rtp[:,2])**2) - (np.sin(rtp[:,2])**2)/(rtp[:,0]**3)

In [11]:
#@njit
def f2im(rtp):
    
    return -2 * (np.sin(rtp[:,1])**2) * np.cos(rtp[:,2]) * np.sin(rtp[:,2]) / (rtp[:,0]**3)

In [12]:
def calc_tau(nframes, coor1, coor2):

    # this is the correlation function
    
    nframes_half = int(np.floor(nframes/2))
    corr = np.zeros((nframes_half))
    
    for tau in range(0, nframes_half):
        

        res = coor1[0:nframes-tau]*coor2[tau:nframes]
        
        corr[tau] = np.sum( res /(nframes-tau) ) # average
        #corr[tau] = np.sum(res)

    return corr

In [12]:
def calc_tau_np(coordinates):
    
    mn = np.mean(coordinates)
    var = np.var(coordinates)
    
    ndata = coordinates - mn
    
    acorr = np.correlate(ndata, ndata, 'full')[len(ndata)-1:]
    acorr = acorr / var / len(ndata)
    
    return acorr

In [35]:
def calculate_t1(traj,id1,id2):
            
    nframes = len(traj)
    
    coor = md.compute_displacements(traj,[[id1,id2]])[:,0,:] #### displacement vector between two hydrogens (A)
                
    spherical_coor = appendSpherical_np(coor)
    
    corr1 = calc_tau(nframes, f1re(spherical_coor),f1re(spherical_coor)) # Eq. 4
    
    #corr1 = calc_tau_np(f1re(spherical_coor))
    
    corr2 = calc_tau(nframes,f1im(spherical_coor),f1im(spherical_coor)) # Eq. 4
    
    #corr2 = calc_tau_np(f1im(spherical_coor))
    
    corr_f1 = corr1 + corr2
    
    corr2_1 = calc_tau(len(traj), f2re(spherical_coor),f2re(spherical_coor)) # Eq. 4
    corr2_2 = calc_tau(len(traj), f2im(spherical_coor), f2im(spherical_coor)) # Eq. 4
    
    #corr2_1 = calc_tau_np(f2re(spherical_coor))
    #corr2_2 = calc_tau_np(f2im(spherical_coor))
    
    corr_f2 = corr2_1 + corr2_2
        
    return [corr1, corr2, corr2_1, corr2_2]

In [14]:
#https://physics.nist.gov/cgi-bin/cuu/Value?gammapp

gamma = 2.675153151*10**8 #  s-1 T-1 

hbar = 1.0545718*10**(-34) # m^2 kg / s

### spce ###

In [15]:
traj = md.load('./spce/build_and_eq/run.xtc', top = 'tip3p/build_and_eq/eq.gro')

/home/batuhan/anaconda3/envs/py37/lib/python3.7/site-packages/mdtraj/formats/gro.py:307: UserWarning: WARNING: two consecutive residues with same number (SOL, HOH)
  warnings.warn("WARNING: two consecutive residues with same number (%s, %s)" % (thisresname, residue.name))


In [17]:
# we should get T1 = 7.0 s #

In [18]:
factor = 9/8*gamma**4*hbar**2
dt = 1*10**(-12) # 1 ps saving frequency

In [28]:
## let's get the intermolecular only (same water) ##

c1_intra = 0;
c2_intra = 0;
c3_intra = 0;
c4_intra = 0;

hydrogens = traj.topology.select('type H')

for resid in range(0,int(len(hydrogens)/2)):
    
    print(resid,end='\r')
    
    [c1_tmp, c2_tmp, c3_tmp, c4_tmp] = calculate_t1(traj[0:100], hydrogens[(resid-1)*2], hydrogens[resid*2])
    
    c1_intra += c1_tmp;
    c2_intra += c2_tmp;
    c3_intra += c3_tmp;
    c4_intra += c4_tmp;
    

In [31]:
## let's get the intermolecular only (different water) ##

from itertools import combinations


c1_inter = 0;
c2_inter = 0;
c3_inter = 0;
c4_inter = 0;

hydrogens = traj.topology.select('type H')

pair_list = list(combinations(hydrogens, 2))

w = 0

for resid in pair_list:
    
    if np.abs(resid[0] - resid[1]) != 1:
    
        [c1_tmp, c2_tmp, c3_tmp, c4_tmp] = calculate_t1(traj[0:100], resid[0], resid[1])
    
        c1_inter += c1_tmp;
        c2_inter += c2_tmp;
        c3_inter += c3_tmp;
        c4_inter += c4_tmp;
        
        w = w + 1
        
        print(str(w) + '/' + str(len(pair_list)),end='\r')
    

KeyboardInterrupt: 

In [36]:
## let's get all ##

from itertools import combinations


c1_all = 0;
c2_all = 0;
c3_all = 0;
c4_all = 0;

hydrogens = traj.topology.select('type H')

pair_list = list(combinations(hydrogens, 2))

w = 0

for resid in pair_list:
    
        [c1_tmp, c2_tmp, c3_tmp, c4_tmp] = calculate_t1(traj[0:100], resid[0], resid[1])
    
        c1_all += c1_tmp;
        c2_all += c2_tmp;
        c3_all += c3_tmp;
        c4_all += c4_tmp;
        
        w = w + 1
        
        print(str(w) + '/' + str(len(pair_list)),end='\r')
        
        if np.mod(w, 1000) == 0:
            
            integrals = (trapezoid(c1_all + c2_all) + trapezoid(c3_all+c4_all))/w; # Eq. 5 with omega = 0
            
            with open("integral.txt", "a") as myfile:
                myfile.write(str(integrals)+'\n')
    

KeyboardInterrupt: 

In [37]:
h = 1.05*10**(-34); gamma = 267.513*10**6; mu = 4*np.pi*10**(-7);
factorT1 = 100; factorT2 = 10;

In [38]:
integrals = (trapezoid(c1_all + c2_all)*factorT1 + trapezoid(c3_all+c4_all)*factorT2)/w;

g1 = (9/8) * (mu/np.pi/4)**2 * gamma**4 * h**2 *2 * integrals * 10**(45)
print(1/g1)

4140.5534463685735
